# import

In [ ]:
%pip install wandb -q
import wandb
import numpy as np
import random
import cv2
from tqdm import tqdm
import pandas as pd
random.seed(1)

# 
# sweep_config={
#     'method' : 'random' ,
#     'metric' : { 'name' : 'val_acc' , 'goal' : 'maximize' } ,
#     'parameters' : {
#         'epochs' : { 'values' : [3,5,7] },
#         'n_hidden_layers' : {'values' : [3,2]},
#         'n_hidden_layer_size' : { 'values' : [32,64,128]},
#         'batch_size' : { 'values' : [32,64]},
#         'weight_decay' : { 'values' : [0.0005]},
#         'learning_rate' : { 'values' : [2e-3,1e-4]},
#         'optimizer' : { 'values' : ['sgd','ngd','mgd','rmsprop','adam','nadam','adagrad'] },
#         'activations' : { 'values' : ['sigmoid','Relu','tanh'] },
#         'loss_function' : {'values' : ['cross_entropy']},
#         'weight_ini' : {'values' : ['random' , 'xavier']}
#     }
# }

# sweep config for loss
sweep_config1={
    'method' : 'random' ,
    'metric' : { 'name' : 'val_acc' , 'goal' : 'maximize' } ,
    'parameters' : {
        'epochs' : { 'values' : [5] },
        'n_hidden_layers' : {'values' : [3]},
        'n_hidden_layer_size' : { 'values' : [32]},
        'batch_size' : { 'values' : [32]},
        'weight_decay' : { 'values' : [0.0005]},
        'learning_rate' : { 'values' : [1e-3]},
        'optimizer' : { 'values' : ['nadam','ngd'] },
        'activations' : { 'values' : ['sigmoid'] },
        'loss_function' : {'values' : ['cross_entropy','squared_error']},
        'weight_ini' : {'values' : ['random']}
    }
}

# sweep_config={
#     'method' : 'random' ,
#     'metric' : { 'name' : 'val_acc' , 'goal' : 'maximize' } ,
#     'parameters' : {
#         'epochs' : { 'values' : [3,7] },
#         'n_hidden_layers' : {'values' : [3,2]},
#         'n_hidden_layer_size' : { 'values' : [16,56]},
#         'batch_size' : { 'values' : [50,70]},
#         'weight_decay' : { 'values' : [0.0003] },
#         'learning_rate' : { 'values' : [2e-3] },
#         'optimizer' : { 'values' : ['ngd','rmsprop','adam','nadam','adagrad'] },
#         'activations' : { 'values' : ['sigmoid','tanh'] },
#         'loss_function' : {'values' : ['cross_entropy' , 'squared_error']},
#         'weight_ini' : {'values' : ['random' , 'xavier']}
#     }
# }

# sweep_config={
#     'method' : 'random' ,
#     'metric' : { 'name' : 'val_acc' , 'goal' : 'maximize' } ,
#     'parameters' : {
#         'epochs' : { 'values' : [12,7] },
#         'n_hidden_layers' : {'values' : [3,2]},
#         'n_hidden_layer_size' : { 'values' : [16,56]},
#         'batch_size' : { 'values' : [50,100]},
#         'weight_decay' : { 'values' : [0.0003] },
#         'learning_rate' : { 'values' : [2e-3] },
#         'optimizer' : { 'values' : ['ngd','rmsprop','adam','nadam','adagrad'] },
#         'activations' : { 'values' : ['sigmoid','tanh','Relu'] },
#         'loss_function' : {'values' : ['cross_entropy' , 'squared_error']},
#         'weight_ini' : {'values' : ['random' , 'xavier']}
#     }
# }

# sweep_config={
#     'method' : 'bayes' ,
#     'metric' : { 'name' : 'val_acc' , 'goal' : 'maximize' } ,
#     'early_terminate' : {
#         'type' : 'hyperband' ,
#         'min_iter' : 5
#     },
#     'parameters' : {
#         'epochs' : { 'values' : [10] },
#         'n_hidden_layers' : {'values' : [3]},
#         'n_hidden_layer_size' : { 'values' : [32]},
#         'batch_size' : { 'values' : [16]},
#         'weight_decay' : { 'values' : [0.0005] },
#         'learning_rate' : { 'values' : [5e-3] },
#         'optimizer' : { 'values' : ['rmsprop'] },
#         'activations' : { 'values' : ['Relu'] },
#         'loss_function' : {'values' : ['squared_error']},
#         'weight_ini' : {'values' : ['random']}
#     }
# }



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# NN

In [ ]:
class NeuralNetwork():

  def __init__(self,n_input,n_output,n_hidden_layers,n_hidden_neurons):
    self.n_input = n_input
    self.n_output = n_output
    self.n_hidden_layers = n_hidden_layers
    self.n_hidden_neurons = n_hidden_neurons 
    self.layers_size = [self.n_input] + n_hidden_neurons + [self.n_output]  
    self.lambda1 = None 
    self.eta = None
    self.batch_size = None
    self.n_epoch = None
    self.X_train = None
    self.Y_train = None
    self.Y_train_o = None
    self.X_test = None
    self.Y_test = None
    self.Y_test_o = None
    self.vx = None
    self.vy = None
    self.vy_o = None
    self.arguments = None
    
    

  def create_network_random(self):
    # self.total_layers = 2 + self.n_hidden_layers 
    self.W = {}
    self.b = {} 
    for i in range(self.n_hidden_layers+1):
      self.W[i+1] = np.random.randn(self.layers_size[i],self.layers_size[i+1])
      self.b[i+1] = np.reshape(np.random.randn(self.layers_size[i+1]),(1,self.layers_size[i+1]))
    
    # print("W",len(self.W))
    # for i in self.W.values():
    #   print(np.shape(i)) 
    # print("self.b",len(self.b))
    # for i in self.b.values():
    #   print(len(i))

  def create_network_xavier(self):
    # self.total_layers = 2 + self.n_hidden_layers 
    self.W = {}
    self.b = {}
    for i in range(self.n_hidden_layers+1):
      self.W[i+1] = np.random.rand(self.layers_size[i],self.layers_size[i+1]) * np.sqrt(self.layers_size[i]+self.layers_size[i+1])
      self.b[i+1] =  np.zeros((1,self.layers_size[i+1]))
   
  def sigmoid(self,n):
    n[n<-1e+2] = -1e+2
    n[n>1e+2] = 1e+2 
    return 1.0/(1.0 + np.exp(-n))

  # single value n
  def grad_sigmoid(self,n):
    return n * (1 - n)

  def Relu(self,n):
    n[n<0]=0
    n[n>1e+4] = 1e+4
    return n
    
  def grad_Relu(self,n):
    n[n<=0] = 0
    n[n>0] = 1
    return n

  def tanh(self,n):
    n[n<-1e+2] = -1e+2
    n[n>1e+2] = 1e+2 
    return (np.exp(n) - np.exp(-1*n)) / (np.exp(n) + np.exp(-1*n))
  
  def grad_tanh(self,n):
    return 1-np.power(self.tanh(n),2)

  # a is list
  def softmax(self,a):
    a[a>1e+2] = 1e+2
    a[a<-1e+2]= -1e+2
    t = np.exp(a)
    return t/np.sum(t)
  # y_o original output y_p predicted output
  def cross_entropy(self,label,pred):
    t=np.multiply(pred,label)
    t=t[t!=0]
    t=-np.log(t)
    t=t[0]
    # print(yl)
    return t + self.regularize_loss()
  

  #  gradient for the oputput layer when cross entropy is used
  def grad_cross_entropy(self,y_p,y_o):
    # print(type((y_p - y_o) ))
    return (y_p - y_o) 

  def regularize_loss(self):
    total = 0
    for i in self.W.keys():
      total += np.sum(np.square(self.W[i]))
    return self.lambda1 * total

  def squared_error(self,y_o,y_p):
    return  np.sum((np.array(y_o)-np.array(y_p))**2) + self.regularize_loss()

  def grad_squared_error(self,y_o,y_p):
    # print(y_o,y_p)
    y_o = list(y_o)
    y_p = list(y_p)
    temp =[]
    ind = y_o.index(max(y_o))
    for i in range(len(y_o)):
      temp.append(-1* 2 * y_p[ind] * (y_o[i]-y_p[ind]) * (y_o[i]-y_p[i]))
   
    return np.array(temp)

  def forward_pass(self, x):
    self.a = {}
    self.h = {}
    self.h[0] = x.reshape(1,-1)
    for i in range(self.n_hidden_layers):
      # print(np.shape(self.h[i]), np.shape(self.W[i+1]),np.shape(self.b[i+1]))
      self.a[i+1] = np.matmul(self.h[i], self.tW[i+1]) + self.tb[i+1]
      # print(self.a[0])
      self.h[i+1] = getattr(self,self.arguments[0])(self.a[i+1])
    self.a[self.n_hidden_layers+1] = np.matmul(self.h[self.n_hidden_layers], self.tW[self.n_hidden_layers+1]) + self.tb[self.n_hidden_layers+1]
    self.h[self.n_hidden_layers+1] = self.softmax(self.a[self.n_hidden_layers+1])
    return self.h[self.n_hidden_layers+1]


  def gradient(self,x,y):
    self.forward_pass(x)
    self.d_h = {}
    self.d_a = {}
    self.d_a[self.n_hidden_layers+1] = getattr(self,"grad_"+self.arguments[2])(self.h[self.n_hidden_layers+1], y)
    for i in range(self.n_hidden_layers+1, 0, -1):
      # print(np.shape(self.h[i-1]), np.shape(self.d_a[i]))
      self.d_w[i] += np.matmul(self.h[i-1].T, self.d_a[i]) + self.tW[i]*self.lambda1
      self.d_b[i] += self.d_a[i]
      self.d_h[i-1] = np.matmul(self.d_a[i], self.tW[i].T)
      self.d_a[i-1] = np.multiply(self.d_h[i-1],getattr(self,"grad_"+self.arguments[0])(self.h[i-1]))


  # dictionary add key wise
  def add(self,d1,d2,m1=1,m2=1):
    temp ={}
    # print(d1,d2,"dd")
    if (m2==0):
      return d1
    for i in d1.keys():
      temp[i] = m1 * d1[i] + m2 * d2[i]
    return temp

  def mul(self,d1 , m1 = 1):
    temp ={}
    for i in d1.keys():
      temp[i] = m1 * d1[i]
    return temp

  def squr(self,d):
    temp = {}
    for i in d.keys():
      temp[i] = d[i]**2
    return temp

  def adarate(self,d1,d2,n,e):
    temp = {}
    for i in d1.keys():
      temp[i] = (n*d1[i]) /(e+d2[i])**(1/2)
    return temp

  def onehot_encoding(self,a,n_class):
    temp = []
    for i in a:
      t1 = np.zeros(n_class)
      t1[i] = 1
      temp.append(t1)
    return temp

  #  list of classes ex. [1,2,1,..]
  def accuracy(self,y_o, y_p):
    sum = 0
    for i,j in zip(y_o , y_p):
      if(i == j):
        sum = sum + 1
    return sum/len(y_o)

  def predict_and_loss(self,X,Y,n):
    loss = 0
    predicted_class = []
    for i in range(n):
      # forward pass
      pr = self.forward_pass(X[i])[0]
      pt = list(pr)
      predicted_class.append(pt.index(max(pr)))

      #  loss
      # print(i,len(Y))
      # print(p)
      ll = getattr(self,self.arguments[2])(Y[i],pt)
      # print(ll)
      if(np.isnan(ll) or np.isinf(ll)):
        loss = loss + 1e+100
      else:
        loss = loss + ll

    return predicted_class , loss/n

  def accuracy_and_loss(self,X,Y,y):
    n_points , _ = np.shape(X)
    # print(len(Y))
    p2 , l = self.predict_and_loss(X,Y,n_points)
    # print(p)
    acc  = self.accuracy(y, p2)

    return acc , l

  
  

  def sgd(self):
    self.d_w , self.d_b = self.oW , self.ob
    
    for i in tqdm(range(self.n_epoch), total=self.n_epoch, unit="epoch"):
      for j in range(self.n_points):
        # print(j)
        self.gradient(self.X_train[j],self.Y_train[j])

        # for k in range(self.n_hidden_layers+1):
        #   d_w[k] += t1[k]
        #   d_b[k] += t2[k]
      

        if(j%self.batch_size == 0):
          # print(j)
          # print("dw",d_w,"db",d_b)
          for k in self.W.keys():
            self.W[k] -= self.d_w[k] *self.eta / self.n_points
            self.b[k] -= self.d_b[k] *self.eta / self.n_points
          self.tW = self.W
          self.tb = self.b
          self.d_w , self.d_b = self.oW , self.ob
          
        

      # train acc and loss
      t4 , t5 = self.accuracy_and_loss(self.X_train,self.Y_train,self.Y_train_o)
      t4,t5 = float(t4),float(t5)
      # validate
      va , vl = self.accuracy_and_loss(self.vx,self.vy,self.vy_o)
      va,vl = float(va),float(vl)
      wandb.log({'train_acc' : t4 , 'train_loss' : t5 , 'val_acc' : va, 'val_loss': vl })
      # print("epoch",i,"train acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)

  def mgd(self,gamma = 0.5):
    self.d_w , self.d_b = self.oW , self.ob
    
    p_w , p_b =self.oW , self.ob
    v_w , v_b =self.oW , self.ob

    for i in tqdm(range(self.n_epoch), total=self.n_epoch, unit="epoch"):
      for j in range(self.n_points):
        self.gradient(self.X_train[j],self.Y_train[j])
    
        if(j%self.batch_size == 0):
          # print(j)
          for k in self.W.keys():
            v_w[k] = p_w[k]*gamma + self.d_w[k]*self.eta
            v_b[k] = p_b[k] *gamma + self.d_b[k] * self.eta 
            self.W[k] -= v_w[k] /self.n_points
            self.b[k] -= v_b[k] /self.n_points
          
          p_w , p_b = v_w , v_b
          # print("dw",d_w,"db",d_b)
          self.d_w , self.d_b = self.oW , self.ob
          self.tW = self.W
          self.tb = self.b
        
      # train acc and loss
      t4 , t5 = self.accuracy_and_loss(self.X_train,self.Y_train,self.Y_train_o)
      t4,t5 = float(t4),float(t5)
      # validate
      va , vl = self.accuracy_and_loss(self.vx,self.vy,self.vy_o)
      va,vl = float(va),float(vl)
      # wandb.log({'train_acc' : t4 , 'train_loss' : t5 , 'val_acc' : va, 'val_loss': vl })
      print("epoch",i,"train acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)

  def ngd(self,gamma = 0.9):
    self.d_w , self.d_b = self.oW , self.ob
    p_v_w , p_v_b = self.oW , self.ob
    v_w , v_b = self.oW , self.ob
    

    for i in tqdm(range(self.n_epoch), total=self.n_epoch, unit="epoch"):
      for j in range(self.n_points):
        self.gradient(self.X_train[j],self.Y_train[j])
      
        if(j%self.batch_size == 0):
          for k in self.W.keys():
            v_w[k] = p_v_w[k]*gamma + self.d_w[k] *self.eta
            v_b[k] = p_v_b[k]*gamma + self.d_b[k] *self.eta
            self.W[k] -= v_w[k] /self.n_points
            self.b[k] -= v_b[k] /self.n_points
            self.tW[k] = self.W[k] - v_w[k] *gamma
            self.tb[k] = self.b[k] - v_b[k] *gamma 

          self.d_w , self.d_b = self.oW , self.ob
          p_v_w , p_v_b = v_w , v_b

      # train acc and loss
      t4 , t5 = self.accuracy_and_loss(self.X_train,self.Y_train,self.Y_train_o)
      t4,t5 = float(t4),float(t5)
      # validate
      va , vl = self.accuracy_and_loss(self.vx,self.vy,self.vy_o)
      va,vl = float(va),float(vl)
      wandb.log({'train_acc' : t4 , 'train_loss' : t5 , 'val_acc' : va, 'val_loss': vl })
      # print("epoch",i,"train acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)

  def adagrad(self,eps = 1e-8):
    self.d_w , self.d_b = {},{}
    v_w , v_b = {},{}
    for i in range(self.n_hidden_layers+1):
        v_w[i+1] = np.zeros((self.layers_size[i], self.layers_size[i+1]))
        v_b[i+1] = np.zeros((1, self.layers_size[i+1]))
        self.d_w[i+1] = np.zeros((self.layers_size[i], self.layers_size[i+1]))
        self.d_b[i+1] = np.zeros((1, self.layers_size[i+1]))
    # print(v_w)
    
    
    for i in tqdm(range(self.n_epoch), total=self.n_epoch, unit="epoch"):
      for j in range(self.n_points):
        self.gradient(self.X_train[j],self.Y_train[j])

        if(j%self.batch_size == 0):
          for k in self.W.keys():
            # print(self.d_w[k])
            # print(self.d_w[k]**2)
            v_w[k] += np.square(self.d_w[k])
            # print(v_w[k])
            v_b[k] += np.square(self.d_b[k])
            # print(v_w[k])
            self.W[k] -= self.eta  * self.d_w[k] / np.sqrt(eps + v_w[k])
            self.b[k] -= self.eta * self.d_b[k] / np.sqrt(eps + v_b[k])

          
          self.d_w , self.d_b = {},{}   
          for k in range(self.n_hidden_layers+1):
            self.d_w[k+1] = np.zeros((self.layers_size[k], self.layers_size[k+1]))
            self.d_b[k+1] = np.zeros((1, self.layers_size[k+1]))
          self.tW = self.W
          self.tb = self.b

      # train acc and loss
      t4 , t5 = self.accuracy_and_loss(self.X_train,self.Y_train,self.Y_train_o)
      t4,t5 = float(t4),float(t5)
      # validate
      va , vl = self.accuracy_and_loss(self.vx,self.vy,self.vy_o)
      va,vl = float(va),float(vl)
      wandb.log({'train_acc' : t4 , 'train_loss' : t5 , 'val_acc' : va, 'val_loss': vl })
      # print("epoch",i,"train acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)

  def rmsprop(self,eps = 1e-8,beta1=0.9):
    self.d_w , self.d_b = {},{}
    v_w , v_b = {},{}
    for i in range(self.n_hidden_layers+1):
        v_w[i+1] = np.zeros((self.layers_size[i], self.layers_size[i+1]))
        v_b[i+1] = np.zeros((1, self.layers_size[i+1]))
        self.d_w[i+1] = np.zeros((self.layers_size[i], self.layers_size[i+1]))
        self.d_b[i+1] = np.zeros((1, self.layers_size[i+1]))
   
    for i in tqdm(range(self.n_epoch), total=self.n_epoch, unit="epoch"):
      for j in range(self.n_points):
        self.gradient(self.X_train[j],self.Y_train[j])
        
        if(j%self.batch_size == 0):
          for k in self.W.keys():
            v_w[k] = beta1*v_w[k] + (1-beta1) * self.d_w[k]**2 
            v_b[k] = beta1*v_b[k] + (1-beta1) * self.d_b[k]**2
            self.W[k] -= self.d_w[k] *self.eta / (eps + v_w[k])**(1/2)
            self.b[k] -= self.eta * self.d_b[k] / np.sqrt(eps + v_b[k])
          self.d_w , self.d_b = {},{}   
          for k in range(self.n_hidden_layers+1):
            self.d_w[k+1] = np.zeros((self.layers_size[k], self.layers_size[k+1]))
            self.d_b[k+1] = np.zeros((1, self.layers_size[k+1]))
          self.tW = self.W
          self.tb = self.b

      # train acc and loss
      t4 , t5 = self.accuracy_and_loss(self.X_train,self.Y_train,self.Y_train_o)
      t4,t5 = float(t4),float(t5)
      # validate
      va , vl = self.accuracy_and_loss(self.vx,self.vy,self.vy_o)
      va,vl = float(va),float(vl)
      wandb.log({'train_acc' : t4 , 'train_loss' : t5 , 'val_acc' : va, 'val_loss': vl })
      # print("epoch",i,"train acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)

  def adam(self,eps = 1e-8,beta1 = 0.9,beta2 =0.999):
    self.d_w , self.d_b = {},{}
    v_w , v_b = {},{}
    m_w , m_b = {},{}
    m_w_hat ,m_b_hat = {},{}
    v_w_hat ,v_b_hat = {},{}
    for i in range(self.n_hidden_layers+1):
        v_w[i+1] = np.zeros((self.layers_size[i], self.layers_size[i+1]))
        v_b[i+1] = np.zeros((1, self.layers_size[i+1]))
        self.d_w[i+1] = np.zeros((self.layers_size[i], self.layers_size[i+1]))
        self.d_b[i+1] = np.zeros((1, self.layers_size[i+1]))
        m_w[i+1] =  np.zeros((self.layers_size[i], self.layers_size[i+1]))
        m_b[i+1] =  np.zeros((1, self.layers_size[i+1]))
        m_w_hat[i+1] =  np.zeros((self.layers_size[i], self.layers_size[i+1]))
        m_b_hat[i+1] =  np.zeros((1, self.layers_size[i+1]))
        v_w_hat[i+1] =  np.zeros((self.layers_size[i], self.layers_size[i+1]))
        v_b_hat[i+1] =  np.zeros((1, self.layers_size[i+1]))
    
   
    time_stamp = 0
    for i in tqdm(range(self.n_epoch), total=self.n_epoch, unit="epoch"):
      for j in range(self.n_points):
        self.gradient(self.X_train[j],self.Y_train[j])
      
        if(j%self.batch_size == 0):
          for k in self.W.keys():
            m_w[k] = beta1 * m_w[k] +(1-beta1) * self.d_w[k]
            m_b[k] = beta1 * m_b[k] +(1-beta1) * self.d_b[k]
            v_w[k] = beta2 * v_w[k] + (1-beta2) * self.d_w[k]**2
            v_b[k] = beta2 * v_b[k] + (1-beta2) * self.d_b[k]**2
            m_w_hat[k] = m_w[k] * 1/(1-beta1**(time_stamp+1))
            m_b_hat[k] = m_b[k] * 1/(1-beta1**(time_stamp+1))
            v_w_hat[k] = v_w[k] * 1/(1-beta2**(time_stamp+1))
            v_b_hat[k] = v_b[k] * 1/(1-beta2**(time_stamp+1))
            self.W[k] -= m_w_hat[k]*self.eta / (v_w_hat[k] + eps)**(1/2)
            self.b[k] -= self.eta * m_b_hat[k] / np.sqrt(eps + v_b_hat[k])
        
          self.d_w , self.d_b = {},{}   
          for k in range(self.n_hidden_layers+1):
            self.d_w[k+1] = np.zeros((self.layers_size[k],self.layers_size[k+1]))
            self.d_b[k+1] = np.zeros((1, self.layers_size[k+1]))
          time_stamp = time_stamp+1
          self.tW = self.W
          self.tb = self.b

      # train acc and loss
      t4 , t5 = self.accuracy_and_loss(self.X_train,self.Y_train,self.Y_train_o)
      t4,t5 = float(t4),float(t5)
      # validate
      va , vl = self.accuracy_and_loss(self.vx,self.vy,self.vy_o)
      va,vl = float(va),float(vl)
      wandb.log({'train_acc' : t4 , 'train_loss' : t5 , 'val_acc' : va, 'val_loss': vl })
      # print("epoch",i,"train acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)

  def nadam(self,eps = 1e-8,beta1 = 0.9,beta2 =0.999):
    self.d_w , self.d_b = {},{}
    v_w , v_b = {},{}
    m_w , m_b = {},{}
    m_w_hat ,m_b_hat = {},{}
    v_w_hat ,v_b_hat = {},{}
    for i in range(self.n_hidden_layers+1):
        v_w[i+1] = np.zeros((self.layers_size[i], self.layers_size[i+1]))
        v_b[i+1] = np.zeros((1, self.layers_size[i+1]))
        self.d_w[i+1] = np.zeros((self.layers_size[i], self.layers_size[i+1]))
        self.d_b[i+1] = np.zeros((1, self.layers_size[i+1]))
        m_w[i+1] =  np.zeros((self.layers_size[i], self.layers_size[i+1]))
        m_b[i+1] =  np.zeros((1, self.layers_size[i+1]))
        m_w_hat[i+1] =  np.zeros((self.layers_size[i], self.layers_size[i+1]))
        m_b_hat[i+1] =  np.zeros((1, self.layers_size[i+1]))
        v_w_hat[i+1] =  np.zeros((self.layers_size[i], self.layers_size[i+1]))
        v_b_hat[i+1] =  np.zeros((1, self.layers_size[i+1]))
    time_stamp=0

    for i in tqdm(range(self.n_epoch), total=self.n_epoch, unit="epoch"):
      for j in range(self.n_points):
        self.gradient(self.X_train[j],self.Y_train[j])
      
        if(j%self.batch_size == 0):
          for k in self.W.keys():
            
            m_w[k] = beta1 * m_w[k] +(1-beta1) * self.d_w[k]
            m_b[k] = beta1 * m_b[k] +(1-beta1) * self.d_b[k]
            v_w[k] = beta2 * v_w[k] + (1-beta2) * self.d_w[k]**2
            # print(v_w[k])
            v_b[k] = beta2 * v_b[k] + (1-beta2) * self.d_b[k]**2
            m_w_hat[k] = m_w[k] * 1/(1-beta1**(time_stamp+1))
            m_b_hat[k] = m_b[k] * 1/(1-beta1**(time_stamp+1))
            v_w_hat[k] = v_w[k] * 1/(1-beta2**(time_stamp+1))
            v_b_hat[k] = v_b[k] * 1/(1-beta2**(time_stamp+1))
            self.W[k] -= self.eta * (beta1 * m_w_hat[k] + (1-beta1)/(1-beta1**(time_stamp+1)) * self.d_w[k]) / (v_w_hat[k] + eps)**(1/2)
            self.b[k] -= self.eta *(beta1 * m_b_hat[k] + (1-beta1)/(1-beta1**(time_stamp+1)) * self.d_b[k]) / np.sqrt(eps + v_b_hat[k])
            self.tW[k] = self.W[k] - beta1 * m_w[k]
            self.tb[k] = self.b[k] - beta1 * m_b[k]

          self.d_w , self.d_b = {},{}   
          for k in range(self.n_hidden_layers+1):
            self.d_w[k+1] = np.zeros((self.layers_size[k],self.layers_size[k+1]))
            self.d_b[k+1] = np.zeros((1, self.layers_size[k+1]))
          time_stamp = time_stamp +1

      # train acc and loss
      t4 , t5 = self.accuracy_and_loss(self.X_train,self.Y_train,self.Y_train_o)
      t4,t5 = float(t4),float(t5)
      # validate
      va , vl = self.accuracy_and_loss(self.vx,self.vy,self.vy_o)
      va,vl = float(va),float(vl)
      wandb.log({'train_acc' : t4 , 'train_loss' : t5 , 'val_acc' : va, 'val_loss': vl })
      # print("epoch",i,"train acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
        


  def fit(self,x_train,y_train,vx,vy,x_test,y_test,arg,optimizer,weight_ini,batch_size,epoch,lambda1,eta,run):
    self.X_train = x_train
    self.Y_train = self.onehot_encoding(y_train,10)
    self.Y_train_o = y_train
    self.X_test = x_test
    self.Y_test = self.onehot_encoding(y_test,10)
    self.Y_test_o = y_test
    self.vx = vx
    self.vy = self.onehot_encoding(vy,10)
    self.vy_o = vy
    self.arguments = arg
    self.batch_size = batch_size
    self.n_epoch = epoch
    self.lambda1 = lambda1
    self.eta = eta
    self.n_points , _ = np.shape(self.X_train)
    self.ob = {}
    self.oW = {}
    for i in range(self.n_hidden_layers+1):
        self.oW[i+1] = np.zeros((self.layers_size[i], self.layers_size[i+1]))
        self.ob[i+1] = np.zeros((1, self.layers_size[i+1]))
    getattr(self,"create_network_"+weight_ini)()
    
    self.tW = self.W
    self.tb = self.b
    
    getattr(self,optimizer)()
    
    
    n_p_t , _ = np.shape(self.X_test) 
    predicted_classes , test_loss = self.predict_and_loss(self.X_test,self.Y_test,n_p_t)
    # acc = self.accuracy(predicted_classes,self.Y_test_o)
    # dict = {'train_acc' : t1 , 'train_loss' : t2 ,'val_loss' : t4 , 'val_acc' :t3 , 'test_loss':test_loss,'test_acc':acc }
    # # np.save(str(run.config.optimizer)+"_"+str(run.config.loss_function),D)
    # df = pd.DataFrame(dict)
    # df.to_csv(str(run.config.optimizer)+"_"+str(run.config.loss_function)+".csv")

    # np.save("drive/My Drive/DL_assignments/assignment1/weight/"+str(run.name)+"W",self.W)
    # np.save("drive/My Drive/DL_assignments/assignment1/bias/"+str(run.name)+"b",self.b)
    label = ["T-shirt/top" ,"Trouser" ,"Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
    # wandb.sklearn.plot_confusion_matrix(self.Y_test_o, predicted_classes, label)
    
    run.finish()
    return 

    





# ld

In [ ]:



from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
def load_data():
 
  (X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
  # flatten data
  x_train = []
  for j in X_train:
    x_train.append(j.flatten())
  x_train=np.array(x_train,dtype = np.float32)/255

  # mean centered
  temp = []
  m = np.array(np.mean(x_train,0))
  for j in x_train:
    temp.append(np.array(j)-m)
  x_train = np.array(temp,dtype = np.float32)

  # print(x_train[0] - x_train[1])
  x_test = []
  for j in X_test:
   x_test.append(j.flatten())
  x_test=np.array(x_test,dtype = np.float32)/255

  temp = []
  for j in x_test:
    temp.append(np.array(j)-m)
  x_test = np.array(temp,dtype = np.float32)
  # print(x_train.shape,x_test.shape)

  x_train , vx_test , y_train , vy_test = train_test_split(x_train,Y_train,test_size=0.10,random_state=12)
  kk=10
  return x_train ,y_train, vx_test , vy_test, x_test, Y_test



In [ ]:
def train():
  run = wandb.init()
  c = run.config
  name = "op_"+str(c.optimizer)+"_ac_"+str(c.activations)+"_hl_"+str(c.n_hidden_layers)+"_hls_"+str(c.n_hidden_layer_size)+"_ep_"+str(c.epochs)+"_n_"+str(c.learning_rate)+"_bs_"+str(c.batch_size)+"_wi_"+str(c.weight_ini)
  run.name = name
  print(name)
  
  hn = [c.n_hidden_layer_size]*c.n_hidden_layers  
  hl = c.n_hidden_layers 
  l1 = c.weight_decay
  arg = [c.activations,"softmax",c.loss_function]
  opt = c.optimizer
  ep = c.epochs
  bs = c.batch_size
  lr = c.learning_rate
  wi = c.weight_ini

  x_train,y_train,vx,vy,x_test,y_test= load_data()
  n_points , n_input = np.shape(x_train)

  NN = NeuralNetwork(n_input,10,hl,hn)
  NN.fit(x_train,y_train,vx,vy,x_test,y_test,arg,opt,wi,bs,ep,l1,lr,run)

  return




In [ ]:
sweepid= wandb.sweep(sweep_config1,project="loss",entity ="kkk")
wandb.agent(sweepid,train)

In [ ]:
# for i in sweep_config1.parameters.optimizer.values():
#   s1 = str(i)+"_cross_entropy.npy"
#   s2 = str(i)+"_squared_error.npy"
#   d1 = np.load(s1,allow_pickle=True)
#   d2 = np.load(s2,allow_pickle=True)

p=np.load("nadam_cross_entropy.npy",allow_pickle=True)
# print(p['train_acc'])
# p = dict(p)
print(type(p))

In [ ]:
def trin():
  # run = wandb.init()
  # c = run.config
  # # name = "op_"+str(c.optimizer)+"_ac_"+str(c.activations)+"_l_"+str(c.loss_function)+"_hl_"+str(c.n_hidden_layers)+"_hls_"+str(c.n_hidden_layer_size)+"_ep_"+str(c.epochs)+"_n_"+str(c.learning_rate)+"_bs_"+str(c.batch_size)+"_wi_"+str(c.weight_ini)
  # # run.name = name
  # # print(name)
  
  # hn = [c.n_hidden_layer_size]*c.n_hidden_layers  
  # hl = c.n_hidden_layers 
  # l1 = c.weight_decay
  # arg = [c.activations,"softmax",c.loss_function]
  # opt = c.optimizer
  # ep = c.epochs
  # bs = c.batch_size
  # lr = c.learning_rate
  # wi = c.weight_ini

  hn = [32,32]
  hl=2
  l1=0.001
  arg=["sigmoid","softmax","cross_entropy"]
  opt = "mgd"
  ep = 5
  bs = 64
  lr = 0.001
  wi = "random"
  run = 3

 
  x_train,y_train,vx,vy,x_test,y_test= load_data()
  n_points , n_input = np.shape(x_train)

  NN = NeuralNetwork(n_input,10,hl,hn)
  NN.fit(x_train,y_train,vx,vy,x_test,y_test,arg,opt,wi,bs,ep,l1,lr,run)



  return

trin()
